In [2]:
import numpy as np
import scipy.io as sio

# Load matrix A

In [146]:
dmat = sio.loadmat("simulated_heart.mat")
A = dmat['A']
N, M = A.shape
print(N, M)

28 25


# Generate X

In [147]:
np.random.seed(0)
g = np.random.randn(25)+ 5
u = np.random.uniform(0, 10, 25)
x = 0.2*g + u
x_flat = x.flatten()
Ax = A@x_f
print(x_flat)
print(Ax)

[ 1.54070847  7.25638641  7.31670482  7.61751861 10.81099238  7.62274742
  4.78509669  5.3400481   7.95566819  1.68437442  7.69647587  7.9972334
  3.25603316  2.31359798  4.24305616  4.70384257  7.00078352  5.34498348
 10.94635192  1.84962896  2.5781696   2.7438189   7.70397049  3.38448302
  6.11705865]
[ 1.54070847 14.87913383 19.79827738 25.65864268 31.60164684 12.08677478
 22.89337857  5.23411198  6.11705865 10.81099238  9.30189303 19.51542917
 16.75966145 26.64524889 24.34944732 22.40122988  7.44766147  2.5781696
 24.14194393 29.78331892 28.96174005 32.21761972 24.70511057 34.5423107
 27.38793481 25.50639656 29.84559046 22.52750066]


# Monte-Carlo EM-based ML estimation

In [172]:
x_mc = []
for k in range(100):
    y = np.random.poisson(Ax)
    x_new, diff, mse, step = mle_em(1000, A, y)
    print(f'epoc: {k:2d}, diff: {diff: .4E}, mse: {mse:8.4f}, step: {step}')
    x_mc.append(x_new)

epoc:  0, diff:  9.8053E-06, mse:  13.0085, step: 185
epoc:  1, diff:  9.6705E-06, mse:   8.2611, step: 175
epoc:  2, diff:  9.7512E-06, mse:  12.5532, step: 137
epoc:  3, diff:  9.9436E-06, mse:  14.0769, step: 338
epoc:  4, diff:  9.2441E-06, mse:  12.9914, step: 115
epoc:  5, diff:  9.7383E-06, mse:   9.2951, step: 235
epoc:  6, diff:  9.8310E-06, mse:  11.2124, step: 172
epoc:  7, diff:  9.7353E-06, mse:   8.5386, step: 194
epoc:  8, diff:  9.6574E-06, mse:  12.6682, step: 169
epoc:  9, diff:  9.3699E-06, mse:   9.5852, step: 142
epoc: 10, diff:  9.7931E-06, mse:  10.9851, step: 335
epoc: 11, diff:  9.5951E-06, mse:  11.7150, step: 139
epoc: 12, diff:  9.4666E-06, mse:  11.5947, step: 129
epoc: 13, diff:  9.8679E-06, mse:  13.4083, step: 421
epoc: 14, diff:  9.9157E-06, mse:   8.9436, step: 494
epoc: 15, diff:  9.7105E-06, mse:  11.5126, step: 180
epoc: 16, diff:  1.0940E-05, mse:  13.0435, step: 1000
epoc: 17, diff:  9.6965E-06, mse:  11.6977, step: 107
epoc: 18, diff:  9.9215E-06

# Evaluation by average estimations

In [173]:
x_mcem = np.average(x_mc, axis=0)
print(x_mcem)
print(x_flat)
print(f"mse: {np.linalg.norm(x_mcem - x_flat): .4f}")

[ 1.58514561  7.05321615  7.84218961  7.28920865 11.24756076  7.78068855
  5.26403763  5.13733322  7.19892968  1.57490981  6.4147914   8.0409678
  3.42553386  3.19582516  4.80572169  5.01841024  8.15573494  4.7664087
 10.02424177  1.58800636  2.74167152  2.17529017  7.97295396  3.56951468
  5.80670806]
[ 1.54070847  7.25638641  7.31670482  7.61751861 10.81099238  7.62274742
  4.78509669  5.3400481   7.95566819  1.68437442  7.69647587  7.9972334
  3.25603316  2.31359798  4.24305616  4.70384257  7.00078352  5.34498348
 10.94635192  1.84962896  2.5781696   2.7438189   7.70397049  3.38448302
  6.11705865]
mse:  2.7396


# Functions

In [138]:
def intialize(y):
    x_start = np.zeros(M)
    for k, o in enumerate(y):
        aver = o/sum(A_matrix[k])
        x_start += A_matrix[k]*aver
        # print(k, o, aver)
    x_start = x_start/N
    return x_start

In [139]:
def em(A, y, x_old):
    z = np.zeros((N, M))
    for i in range(N):
        z[i] = y[i]*A[i]*x_old/sum(A[i]*x_old)
    x_new = np.zeros(M)
    for j in range(M):
        x_new[j] = sum(z[:, j])/(sum(A[:, j]))
    return x_new

In [143]:
def mle_em(max_iter, A, y, x_true=x_flat):
    x_old = intialize(y)
    mse = []
    for i in range(max_iter):
        x_new = em(A_matrix, y, x_old)
        mse = np.linalg.norm(x_new-x_true)
        diff = np.linalg.norm(x_new-x_old)
        # if i%100 == 0:
        #     print(f'step: {i}, diff: {diff}, mse: {mse}')
        if diff < 1e-5:
            return x_new, diff, mse, i
        x_old = x_new
    return x_new, diff, mse, max_iter